In [1]:
# Personal 
from data.TrajectorySet import TrajectorySet
from data.Sampler import Sampler 
from data.DatasetCL import DatasetCL
from models.cl_model import mlpCL
from trainer.cl_trainer import train_cl
from utils.visualizations import visualize_embeddings
from utils.tensor_utils import split_data

# Misc
import minari 
import os

# Torch 
import torch 

# PyTorch Lightning 
import pytorch_lightning
from pytorch_lightning.loggers import WandbLogger

In [2]:
MINARI_DATASET = minari.load_dataset("D4RL/pointmaze/large-v2")
PROJECT_ROOT = os.getcwd() 
CHECKPOINT_PATH = PROJECT_ROOT + "/saved_models"

PROJECT_NAME = "Contrastive Learning RL"
RUN_NAME = "best_model_uniform_15_new"
FILENAME = RUN_NAME

CONFIG = {
        "distribution": "u",
        "num_state_pairs": 1_000_000,
        "batch_size": 4096,
        "lr": 1e-3,
        "weight_decay": 1e-5, 
        "temperature": 30,
        "max_epochs": 1000,
        "filename": FILENAME,
        "device": "mps" if torch.backends.mps.is_available() else "cuda" if torch.cuda.is_available() else "cpu"
    }

In [3]:
wandb_logger = WandbLogger(
        project=PROJECT_NAME, 
        name=RUN_NAME, 
        save_dir = PROJECT_ROOT, 
        log_model=True,
        config = CONFIG) 

dist = CONFIG["distribution"]
num_state_pairs = CONFIG["num_state_pairs"]
batch_size = CONFIG["batch_size"]
lr = CONFIG["lr"]
weight_decay = CONFIG["weight_decay"]
temperature = CONFIG["temperature"]
max_epochs = CONFIG["max_epochs"]
device = CONFIG["device"]
filename = CONFIG["filename"]

In [4]:
T = TrajectorySet(dataset=MINARI_DATASET)
S = Sampler(T, dist=dist, b=15, sigma=15)

split_val = 0.8 
train_batch = int(round(num_state_pairs * split_val))
val_batch = int(round(num_state_pairs * (1 - split_val)))

train_dataset = DatasetCL(S, num_state_pairs=train_batch)
val_dataset = DatasetCL(S, num_state_pairs=val_batch)

print(train_batch)
print(val_batch)

800000
200000


In [5]:
MINIBATCH_SIZE = batch_size
print("Minibatch size:", MINIBATCH_SIZE)
print(type(MINIBATCH_SIZE))

Minibatch size: 4096
<class 'int'>


In [ ]:
model = train_cl(cl_model=mlpCL, 
                train_ds=train_dataset, 
                val_ds = val_dataset,
                batch_size= MINIBATCH_SIZE,
                logger=wandb_logger, 
                checkpoint_path=CHECKPOINT_PATH,
                max_epochs=max_epochs,
                filename=filename,  
                device = device, 
                lr=lr, 
                temperature=temperature, 
                weight_decay = weight_decay,
                )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3060') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
wandb: Currently logged in as: ray-s (ray-s-university-of-alberta) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:654: Checkpoint directory C:\Users\ray\Documents\2025 RA\contrastive-learning-RL\saved_models exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type       | Params | Mode 
--------------------------------------------
0 | mlp  | Sequential | 44.5 K | train
--------------------------------------------
44.5 K    Trainable params
0         Non-trainable params
44.5 K    Total params
0.178     Total estimated model params size (MB)
8         Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


c:\Users\ray\AppData\Local\anaconda3\envs\CL_RL_Cuda\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 829:  16%|█▋        | 32/195 [00:01<00:07, 21.10it/s, v_num=bj2z, train/nll_loss_step=7.490, train/top1_step=0.0243, train/top5_step=0.0695, val/nll_loss=7.530, val/top1=0.0225, val/top5=0.0651, train/nll_loss_epoch=7.510, train/top1_epoch=0.0227, train/top5_epoch=0.066]     